# About this Jupyter Notebook

@author: Yingding Wang

This notebook is originated from the example of kubeflow visual metric with KF Pipeline UI (https://github.com/kubeflow/pipelines/blob/master/samples/test/metrics_visualization_v1.py)

In [1]:
import sys
!{sys.executable} -m pip install --upgrade --user kfp==1.8.12

In [7]:
import kfp.dsl
import kfp.v2.dsl.component 

@component(
    packages_to_install=['sklearn'],
    base_image='python:3.9'
)
def iris_sgdclassifier(test_samples_fraction: float, metrics: Output[ClassificationMetrics]):
    from sklearn import datasets, model_selection
    from sklearn.linear_model import SGDClassifier
    from sklearn.metrics import confusion_matrix

    iris_dataset = datasets.load_iris()
    train_x, test_x, train_y, test_y = model_selection.train_test_split(
        iris_dataset['data'], iris_dataset['target'], test_size=test_samples_fraction)


    classifier = SGDClassifier()
    classifier.fit(train_x, train_y)
    predictions = model_selection.cross_val_predict(classifier, train_x, train_y, cv=3)
    metrics.log_confusion_matrix(
        ['Setosa', 'Versicolour', 'Virginica'],
        confusion_matrix(train_y, predictions).tolist() # .tolist() to convert np array to list.
    )

@dsl.pipeline(
    name='metrics-visualization-pipeline')
def metrics_visualization_pipeline():
    iris_sgdclassifier_op = iris_sgdclassifier(test_samples_fraction=0.3)

ModuleNotFoundError: No module named 'kfp.v2.dsl.component'

In [10]:
NAME_SPACE="kubeflow-kindfor"
EXPERIMENT_NAME="kf-metrics"

client = kfp.Client()
client.set_user_namespace(NAME_SPACE)

In [11]:
client.create_run_from_pipeline_func(pipeline_func=confusion_matrix_pipeline, arguments=None,
                                     experiment_name=EXPERIMENT_NAME, namespace=NAME_SPACE,
                                     pipeline_conf=dsl.PipelineConf(),
                                     mode = dsl.PipelineExecutionMode.V1_LEGACY
                                    )

RunPipelineResult(run_id=722520a5-c399-493b-a25d-b594c7f13783)